In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:Data Import
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
pip install category_encoders

# **Data Import**

In [ ]:
import pandas as pd
df_train = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
df_test = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")

df_train = df_train.set_index('Id')
df_test = df_test.set_index('ForecastId')
print("Size-train:", df_train.shape[0], "test:", df_test.shape[0])
df_train.head()

In [ ]:
df_train = df_train.fillna("MISSING")
df_test = df_test.fillna("MISSING")

# Data Understanding

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
print("TRAIN", df_train.isna().sum())
print("TEST", df_test.isna().sum())

# Data Cleaning & Encoding

In [ ]:
#Split dates into Month, Day columns and remove year since all observations are from 2020
from datetime import datetime, timedelta
df_train.Date = [datetime(year=int(d[0:4]), month=int(d[6:7]), day=int(d[8:10])) for d in df_train.iloc[:, 2]]
df_test.Date = [datetime(year=int(d[0:4]), month=int(d[6:7]), day=int(d[8:10])) for d in df_test.iloc[:, 2]]
df_train.Date = df_train.Date.astype(int)
df_test.Date = df_test.Date.astype(int)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_train.Country_Region = le.fit_transform(df_train.Country_Region)
df_train.Province_State = le.fit_transform(df_train.Province_State)
df_test.Country_Region = le.fit_transform(df_test.Country_Region)
df_test.Province_State = le.fit_transform(df_test.Province_State)

# Modeling

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
from xgboost import XGBRegressor
#Prepare X and y for training and testing data
y_case_train = df_train.ConfirmedCases
y_fatal_tain = df_train.Fatalities

X_train = df_train.iloc[:, 0:3]

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
import numpy as np
from sklearn.metrics import r2_score

countries = df_train.Country_Region.unique()
for country in countries:
  states = df_train.Province_State[df_train.Country_Region == country].unique()
  for state in states:
    gbr_case = XGBRegressor(n_estimators=10)
    gbr_case.fit(X_train, y_case_train)
    #reg = LinearRegression()
    #reg.fit(X_train, y_case_train)
    
    gbr_fatal = XGBRegressor(n_estimators=10)
    gbr_fatal.fit(X_train, y_fatal_tain)
    #reg1 = LinearRegression()
    #reg1.fit(X_train, y_case_train)
    
    #Will change it for testing
    
    ypred_case = gbr_case.predict(df_test)
    ypred_fatal = gbr_fatal.predict(df_test)
    #ypred_case = reg.predict(df_test)
    #ypred_fatal = reg1.predict(df_test)
    #rmse = rmsle(y_case_train, ypred_case)
    #print(rmse)

In [ ]:
ForecastId = df_test.index


finaldata = pd.DataFrame({'ForecastId': ForecastId, 'ConfirmedCases': ypred_case, 'Fatalities': ypred_fatal})
output =pd.DataFrame()
output = pd.concat([output, finaldata], axis=0)

In [ ]:
output.ForecastId = output.ForecastId.astype('int')
output.tail()
output.to_csv('submission.csv', index=False)